# Simple ETL Application

In this notebook, we connect to an api, extract data, create a spark dataframe and then finally write to a postgres database. 

In [2]:
import requests
import json
import pandas as pd
import psycopg2
from src.get_data import get_data

## Get data from Randomuser API

In [4]:
def get_data():
    """
    get data from random user api
    """
    try:
        url = "https://randomuser.me/api/"
        response = requests.get(url)
        status_code = response.status_code
        response = json.loads(response.text)
        return status_code,response['results'][0]
    except Exception as e:
        print(f"there was an error{e}")
        return None

## Create dataframe

In [10]:
people_list = []

for _ in range(5):
    _,data = get_data()
    people_list.append(data)

data = pd.DataFrame(people_list,index=None)

In [11]:
data

,gender,name,location,email,login,dob,registered,phone,cell,id,picture,nat
0,female,"{'title': 'Miss', 'first': 'Mayla', 'last': 'L...","{'street': {'number': 4020, 'name': 'Heidereed...",mayla.liemburg@example.com,{'uuid': 'a6076a89-293d-4352-9c76-d6664f3cbda4...,"{'date': '1966-01-24T15:24:22.377Z', 'age': 58}","{'date': '2010-11-28T22:46:07.877Z', 'age': 13}",(091) 7686296,(06) 50546414,"{'name': 'BSN', 'value': '93561213'}",{'large': 'https://randomuser.me/api/portraits...,NL
1,female,"{'title': 'Miss', 'first': 'Jennie', 'last': '...","{'street': {'number': 7636, 'name': 'Thornridg...",jennie.sutton@example.com,{'uuid': '7e33cdb1-1985-4dc7-b766-5e108ac0a94d...,"{'date': '1951-04-13T15:46:01.575Z', 'age': 73}","{'date': '2003-01-17T18:13:29.755Z', 'age': 21}",(770) 752-8143,(375) 570-8418,"{'name': 'SSN', 'value': '072-98-3114'}",{'large': 'https://randomuser.me/api/portraits...,US
2,male,"{'title': 'Mr', 'first': 'Gaëtan', 'last': 'Ar...","{'street': {'number': 3766, 'name': 'Rue Laure...",gaetan.arnaud@example.com,{'uuid': '50c6e6be-1a64-4995-9ef6-931101f6165e...,"{'date': '1956-03-03T06:45:28.243Z', 'age': 68}","{'date': '2004-02-22T05:32:16.220Z', 'age': 20}",01-09-30-43-99,06-36-01-83-55,"{'name': 'INSEE', 'value': '1560263081013 78'}",{'large': 'https://randomuser.me/api/portraits...,FR
3,male,"{'title': 'Mr', 'first': 'سورنا', 'last': 'علی...","{'street': {'number': 5189, 'name': 'میدان توح...",swrn.aalyzdh@example.com,{'uuid': '7748b26c-0c82-49b3-af91-281e4cd718c7...,"{'date': '1974-07-10T09:28:56.300Z', 'age': 49}","{'date': '2003-12-03T06:44:21.367Z', 'age': 20}",027-84057482,0954-716-0147,"{'name': '', 'value': None}",{'large': 'https://randomuser.me/api/portraits...,IR
4,female,"{'title': 'Mrs', 'first': 'ستایش', 'last': 'سا...","{'street': {'number': 9094, 'name': 'دکتر مفتح...",stysh.slry@example.com,{'uuid': '748be4ff-1274-4e8e-aed3-269d41c4fdc9...,"{'date': '1980-01-02T13:38:15.523Z', 'age': 44}","{'date': '2019-09-26T19:51:34.228Z', 'age': 4}",017-29875773,0935-148-9601,"{'name': '', 'value': None}",{'large': 'https://randomuser.me/api/portraits...,IR


## Create table in postgres Database using psycopg2 library

In [37]:
query = """CREATE TABLE test_table (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    email TEXT UNIQUE
)"""

In [38]:
# url
url = "http://postgres-db:45432/postgres_db"
#user
username = "postgres"
password = "postgres"

conn_details = psycopg2.connect(
    host='postgres-db',
    database="postgres_db",
    user=username,
    # secretlint-disable
    password=password,
    # secretlint-enable
    port='5432'
)

with conn_details:
    cursor = conn_details.cursor()
    query = query
    cursor.execute(query)

DuplicateTable: relation "test_table" already exists


## Insert Records

## Create a spark Session

In [39]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.driver.extraClassPath", "postgresql-connector-java.jar") \
    .config("spark.executor.extraClassPath", "postgresql-connector-java.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/11 21:19:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Create Spark dataframe

In [40]:
spark_df = spark.createDataFrame(data)

## Test Functions 

In [43]:
import pytest

def test_test_data():
    """
    Tests API is returning values
    """
    status_code,_ = get_data()
    
    assert status_code == 200



In [53]:
! pytest

============================= test session starts ==============================
platform linux -- Python 3.11.9, pytest-8.1.1, pluggy-1.4.0
rootdir: /app
plugins: anyio-4.3.0
collected 1 item                                                               

tests/test_get_data.py .                                                 [100%]

============================== 1 passed in 1.34s ===============================
